In [21]:
# %pip install geopy
# %pip install geocoder

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from geopy.geocoders import Nominatim
import numpy as np

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)
plt.rcParams['axes.unicode_minus']=False


In [69]:
df = pd.read_csv('vaccination_all_tweets.csv')
df[['date', 'user_created']] = df[['date', 'user_created']].apply(pd.to_datetime, format='mixed')
df.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [70]:
def get_df_summary():
    print(f'Data Shape: {df.shape}')
    df_desc = pd.DataFrame(df.describe(include='all')).transpose()
    summary = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summary['NA Count'] = df.isnull().sum()
    summary['NA %'] = df.isnull().sum() / len(df) * 100
    summary['Unique Values'] = df.nunique()
    summary['Min'] = df_desc['min'].values
    summary['Max'] = df_desc['max'].values
    summary['Mean'] = df_desc['mean'].values
    summary['Skewness'] = df.skew(numeric_only=True)
    return summary


get_df_summary()

Data Shape: (228207, 16)


,Data Type,NA Count,NA %,Unique Values,Min,Max,Mean,Skewness
id,int64,0,0.000000,228207,1337727767551553536.0,1463250536954830848.0,1407541268918201856.0,-0.023492
user_name,object,2,0.000876,85549,NaN,NaN,NaN,NaN
user_location,object,66911,29.320310,25458,NaN,NaN,NaN,NaN
user_description,object,17023,7.459456,84002,NaN,NaN,NaN,NaN
user_created,datetime64[ns],0,0.000000,86739,2006-07-15 21:06:00,2021-11-22 23:11:26,2015-12-26 09:15:30.594736384,NaN
user_followers,int64,0,0.000000,22439,0.0,16353048.0,96053.927285,12.824363
user_friends,int64,0,0.000000,8196,0.0,582461.0,986.736204,40.744820
user_favourites,int64,0,0.000000,38744,0.0,1299600.0,12104.173018,9.318966
user_verified,bool,0,0.000000,2,NaN,NaN,NaN,2.996087
date,datetime64[ns],0,0.000000,222093,2020-12-12 11:55:28,2021-11-23 20:58:08,2021-06-23 03:29:23.522911232,NaN


In [91]:
def get_unique_values(col):
    # unique_val = pd.DataFrame(columns=['Col', 'Unique Values'])
# for col in df:
    # unique_val.loc[len(unique_val.index)] = [col, list(df[col].unique())]
    # display(unique_val.set_index('Col'))
    # result = df[col].value_counts(dropna=True).rename_axis('unique_values').to_frame('counts')
    result = dict(df[col].value_counts(dropna=True))
    print("{:<30} {:<10} ".format('Values','Count'))
    for key, val in result.items():
        print("{:<30} {:<10} ".format(key,val))
    # return result

In [92]:
get_unique_values('user_location')

Values               Count      
nan                       66911      
Bengaluru India           19656      
India                     8480       
New Delhi India           3176       
Toronto Canada and Worldwide 2653       
United States             2054       
Mumbai India              1993       
Beijing China             1397       
New Delhi                 1288       
Sri Lanka                 1157       
Mumbai                    1056       
Canada                    1030       
London                    983        
Los Angeles CA            976        
Toronto Ontario           960        
London England            935        
Hyderabad India           874        
                          867        
Moscow Russia             821        
Chennai India             760        
California USA            758        
USA                       710        
Malaysia                  679        
Beijing                   677        
United Kingdom            672        
Washington DC 

In [73]:
get_unique_values('text')

{'#COVAXIN vaccine approved for children aged 2 - 18 ... Another milestone in handling #COVID19 ... Well done scientists .': 18,
 'Got my second dose of the #Moderna #vaccine today!! 💉💉': 16,
 "@naashonomics #Covaxin #Ocgn #OCGN #DeltaVariant $ocgn LETS KICK THE DELTA VARIANT'S ASS IN EVERY COUNTRY!!!": 11,
 '@POTUS Approve #COVAXIN': 8,
 'India gets covid vaccine for kids.\n#Covaxin approved for kids aged 2-18.': 7,
 'Big News: \n#Covaxin approved for kids 2-18': 6,
 'BREAKING NEWS :  #SputnikV approved for emergency use in India.': 6,
 '560057, \nAGE 45+\n0-D1, 44-D2 slots,\n#COVAXIN on 18-06-2021\n@ NEL COVAXIN(Free) #BBMP': 5,
 '560076, \nAGE 45+\n99-D1, 100-D2 slots,\n#COVAXIN on 27-07-2021\n@ Rainbow Royal Meenakshi MallC1(1410Rs) #BBMP': 5,
 '560027, \nAGE 45+\n99-D1, 100-D2 slots,\n#COVAXIN on 22-07-2021\n@ Agadi HOSPITAL AND RESEARCH(1410Rs) #BBMP': 5,
 '@POTUS I’ll get vaccinated when you allow #Covaxin to be used in the USA. We need alternatives especially those that work.':

In [74]:
df.fillna(np.NAN)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228202,1460170772299665408,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 09:00:15,45+ #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:30PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose1:… https://t.co/F0WeHxW7oe,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,0,False
228203,1460163268262051841,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:26,"18-44 #BBMP #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFREE Slots \n#COVISHIELD - Dose1:1676, Dose2:… https://t.co/3k8SHHcVVd","['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,1,False
228204,1460163224221851655,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:15,18-44 #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose… https://t.co/INCb38UKmA,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,0,False
228205,1460156376995573765,Gatti Valentino🐾,Southern Africa,"Entrepreneur, self taught cook🍲🌮 @Chelsea @Ferrari, @hornets political activist, London is Blue 💙, Property developer, Aries♈",2019-08-28 10:31:43,8103,3113,45726,False,2021-11-15 08:03:03,They promote their Vaccines leaving out the stronger @Russia vaccine \n#SputnikV https://t.co/nD1J8NvqfA,['SputnikV'],Twitter for Android,

In [75]:
# df['processed_text'] = df['text']
df.insert(11, 'processed_text', df['text'])
def clean_text(text):
    text = str(text).lower() #lower case
    text = re.sub('\n+', ' ', text) #remove split lines
    text = re.sub('http\S+|www|https\S+', '', text) #remove url
    text = re.sub('[#|@]\w+', '', text) #remove hashtags/tags
    text = re.sub('[^\w|\s]|[_|]|[\d+]', '', text) #remove non-word char, symbols, emoji, digits
    text = re.sub('\s+', ' ', text) #remove extra spaces
    text = text.strip() #remove trailing spaces
    return text

# print(clean_text('☆😄_kjlkaj3 nk@ll -.,/;p[p[]\';)(*&^%$#@!.=-~`|}{];/"":<>,.})]')) #sample
print(df['processed_text'][301]) #preprocessing
df['processed_text'] = df['processed_text'].apply(lambda x:clean_text(x)) #cleaning
df['processed_text'][300:2000] #post processing
df

I am beyond relieved to share that I’ll be among the first to receive the #CovidVaccine from #PfizerBioNTech. Thank… https://t.co/1py4jUhGzU


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,processed_text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,same folks said daikon paste could treat a cytokine storm,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",while the world has been on the wrong side of history this year hopefully the biggest vaccination effort weve ev,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,russian vaccine is created to last years,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",facts are immutable senator even when youre not ethically sturdy enough to acknowledge them you were born i,NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,explain to me again why we need a vaccine,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228202,1460170772299665408,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 09:00:15,45+ #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:30PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose1:… https://t.co/F0WeHxW7oe,availability for at pm free slots paid slots dose,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,0,False
228203,1460163268262051841,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:26,"18-44 #BBMP #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFREE Slots \n#COVISHIELD - Dose1:1676, Dose2:… https://t.co/3k8SHHcVVd",availability for at pm free slots dose dose,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,1,False
228204,1460163224221851655,VaxBLR,"Bengaluru, India","Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:15,18-44 #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose… https://t

In [78]:
antarctica = ['Antarctica']
argentina = ['Argentina']
australia = ['Australia', 'Melbourne', 'Sydney', 'New Zealand', '土澳']
austria = ['Austria']
bangladesh = ['Bangladesh']
belgium = ['Belgium', 'Brussels']
brazil = ['Brasil']
cambodia = ['Cambodia']
canada = ['Canada', 'Toronto',  'Vancouver', 'Alberta', 'Ontario', 'Montreal', 'Manitoba']
chile = ['Chile']
china = ['China', 'Beijing', '北京']
cyprus = ['Cyprus']
czechia = ['Czech Republic']
denmark = ['Danmark']
france = ['France', 'Provence-Alp', 'Paris']
germany = ['Germany', 'Heidelberg', 'Berlin', 'Deutschland']
greece = ['Greece']
hongkong = ['Hong Kong', '香港']

india = ['. भारत', 'Ahmedabad', 'Assam', 'Bangalore', 'Bengaluru', 'Bharat', 'Bharat', 'Bhubaneswar', 'Chandigarh', 'Chennai', 'Coimbatore',
                'Delhi', 'Gurgaon', 'Guwahati', 'Hindustan', 'Hyderabad', 'Hyderabad', 'India', 'Jammu And Kashmir', 'Kerala', 'Kharar', 'Kolkata',
                'Kolkata', 'Lucknow', 'Mumbai', 'New Delhi', 'Noida', 'Odisha', 'Odisha', 'Pune', 'Punjab', 'patna', 'आर्यावर्त']

indonesia = ['Indonesia']
ireland = ['Ireland', 'Dublin']
israel = ['Israel']
italy = ['Italy', 'Italia', 'Lazio']
japan = ['Japan', '仙台市', '多摩川のほとり', '新潟市', '日本', '東京', '松戸市', '横須賀市神奈川県', '福岡']
kenya = ['Kenya', 'Nairobi']
korea = ['Korea', '韩国', '대한민국', '서울']
lebanon = ['Lebanon']
malaysia = ['Malaysia', 'malaysia', 'Penang', 'Pinang', 'Petaling Jaya', 'Negeri Sembilan', 'Kuala Lumpur', 'Selangor', 'WIlayah Persekutuan', '馬來西亞']
myanmar = ['Myanmar']
nederland = ['Nederland']
nigeria = ['Nigeria']
pakistan = ['Pakistan', 'Islamabad', 'Karachi']
philippines = ['Philippines', 'Manila', 'Central Luzon', 'Quezon', 'Makati City']
qatar = ['Qatar']
serbia = ['Republic of Serbia']
russia = ['Russia', 'Россия']
singapore = ['Singapore', '新加坡']
slovakia = ['Slovakia']
south_africa = ['South Africa', 'Southern Africa', 'Zimbabwe', 'Johannesburg', 'S/AFRICA']
sri_lanka = ['Sri Lanka', 'Colombo']
switzerland = ['Switzerland']
taiwan = ['Central News Agency', '台灣', '千葉','新北市板橋區', '桃園市', 'Taoyuan City']
thailand = ['Thailand', 'Bangkok', 'กรงเทพมหานคร ประเทศไทย']
netherlands = ['The Netherlands']
trinidad_tobago = ['Trinidad and Tobago', 'Trinidad & Tobago']
turkey = ['Türkiye', 'Turkey']
turkey_israel = ['Turkey-Israel']
us_california = ['Sacramento']
us_colorado = ['Denver']
us_florida = ['Fort Lauderdale',  'Orlando', 'Tampa']
us_illinois = ['Naperville']
us_missouri = ['Jefferson City']
us_new_jersey = ['Cranbury', 'Princeton']
us_north_calorina = ['Charlotte', 'Raleigh']
us_ohio = ['Cleveland', 'Columbus']
us_oregon = ['Portland']
us_tennessee = ['Memphis', 'Nashville']
ukraine = ['Ukraine']
united_arab_emirates = ['United Arab Emirates', 'Dubai']
uk = ['England', 'United Kingdom', 'UK', 'U.K', 'Watford', 'Wales', 'Berkshire', 'Manchester', 'Birmingham', 'Scotland', 'Nottingham']

us = ['535 Radio Lane, Henderson', 'America', 'Atlanta', 'Baltimore', 'Boston', 'CA', 'California', 'Charleston', 'Chicago', 'Cleveland',
            'Colorado', 'Columbus', 'Coral Gables', 'Cranbury', 'Dallas', 'Denver', 'Earth', 'Estados Unidos', 'Everywhere! ', 'Florida', 'Global',
            'Jefferson City', 'Kentucky', 'LA', 'Labelle', 'Las Vega', 'Los Angeles', 'Lufkin', 'Manhattan', 'Medford', 'Memphis, TN', 'Miami', 'Michigan',
            'Minneapolis', 'NY', 'NYC', 'NYC', 'Naperville', 'Nashville, TN | Tucson', 'Naval Base Kitsap', 'New Jersey', 'New Jersey', 'New Orleans',
            'New Orleans, Louisiana', 'New York', 'Okolona', 'Orlando', 'Pewee Valley', 'Philadelphia', 'Phoenix', 'Pittsburgh', 'Planet Location Unknown',
            'Portland', 'Princeton', 'Raleigh, NC', 'Sacramento', 'San Francisco', 'San Jose', 'Seattle', 'Shoreview', 'Southern Cal', 'Stuck in the Middle',
            'TX', 'Tampa', 'Texas', 'Texas', 'Text RESIST to 50409', 'Tucson', 'US', 'USA', 'United States', 'Washington', 'West Palm Beach', 'Worldwide',
            'astroworld']


# full_list = np.concatenate([india_list, us_list, uk_list, canada_list, australia_list, philippines_list, south_africa_list])
# print(*full_list)


In [79]:
map = {'Antarctica':antarctica, 'Argentina':argentina, 'Australia':australia, 'Austria':austria, 'Bangladesh':bangladesh, 'Belgium':belgium,
'Brazil':brazil, 'Cambodia':cambodia, 'Canada':canada, 'Chile':chile, 'China':china,
'Cyprus':cyprus, 'Czech Republic':czechia, 'Denmark':denmark, 'France':france, 'Germany':germany, 'Greece':greece, 'Hong Kong':hongkong, 'India':india,
'Indonesia':indonesia, 'Ireland':ireland, 'Israel':israel, 'Italy':italy, 'Japan':japan,
'Kenya':kenya, 'Korea':korea, 'Lebanon':lebanon, 'Malaysia':malaysia, 'Myanmar':myanmar, 'Nederland':nederland, 'Nigeria':nigeria, 'Pakistan':pakistan,
'Philippines':philippines, 'Qatar':qatar, 'Republic of Serbia':serbia,
'Russia':russia, 'Singapore':singapore, 'Slovakia':slovakia, 'South Africa':south_africa, 'Sri Lanka':sri_lanka, 'Switzerland':switzerland, 'Taiwan':taiwan,
'Thailand':thailand, 'The Netherlands':netherlands, 'Trinidad and Tobago':trinidad_tobago, 'Turkey':turkey,
'Turkey-Israel':turkey_israel, 'US of California':us_california, 'US of Colorado':us_colorado, 'US of Florida':us_florida, 'US of Illinois':us_illinois,
'US of Missouri':us_missouri, 'US of New Jersey':us_new_jersey, 'US of North Calorina':us_north_calorina,
'US of Ohio':us_ohio, 'US of Oregon':us_oregon, 'US of Tennessee':us_tennessee, 'Ukraine':ukraine, 'United Arab Emirates':united_arab_emirates,
'United Kingdom':uk, 'United States':us}

In [80]:
def clean_location(location):
    location = str(location).strip()
    location = re.sub('\s+', ' ', location)
    location = re.sub('[^\w|\s|-]|[\d|\_|\|]', '', location)
    # location = re.sub('[^\-|\,]', '', location)
    # location = re.sub(r'\s+', ' ', location)
    # print(location)
    return location

# print(clean_location('kjl!@#$%^&*()_+-=\{\}|:"<>?~`[]\;,./🇦🇲 12132   cjanldja kjk-kklj⍺')) #sample

In [81]:
df['user_location'] = df['user_location'].apply(lambda x:clean_location(str(x)))
df.insert(3, 'country', df['user_location'])
df

,id,user_name,user_location,country,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,processed_text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,La Crescenta-Montrose CA,La Crescenta-Montrose CA,"Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,same folks said daikon paste could treat a cytokine storm,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,San Francisco CA,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",while the world has been on the wrong side of history this year hopefully the biggest vaccination effort weve ev,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,russian vaccine is created to last years,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,Vancouver BC - Canada,Vancouver BC - Canada,"Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",facts are immutable senator even when youre not ethically sturdy enough to acknowledge them you were born i,NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,nan,nan,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,explain to me again why we need a vaccine,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228202,1460170772299665408,VaxBLR,Bengaluru India,Bengaluru India,"Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 09:00:15,45+ #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:30PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose1:… https://t.co/F0WeHxW7oe,availability for at pm free slots paid slots dose,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,0,False
228203,1460163268262051841,VaxBLR,Bengaluru India,Bengaluru India,"Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:26,"18-44 #BBMP #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFREE Slots \n#COVISHIELD - Dose1:1676, Dose2:… https://t.co/3k8SHHcVVd",availability for at pm free slots dose dose,"['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,1,False
228204,1460163224221851655,VaxBLR,Bengaluru India,Bengaluru India,"Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:15,18-44 #URB

In [82]:
# c = ['Vancouver, BC - Canada', 'La Crescenta-Montrose, CA', 'Your Bed', np.NAN]
# print(len(df['country']))
# print(df['country'][0])
for i in range(len(df['country'])):
   print(f'Processing {df["country"][i]}...')
   country_name = [key for key, val in map.items() if any(substr.lower() in df['country'][i].lower().split() for substr in val)]
   # print(i, country_name)
   if(len(country_name) > 0):
      df.loc[i, 'country'] = country_name[0]

df['country']

228207
La Crescenta-Montrose CA


0           United States
1           United States
2                Your Bed
3                  Canada
4                     nan
               ...       
228202              India
228203              India
228204              India
228205    Southern Africa
228206              India
Name: country, Length: 228207, dtype: object

In [34]:
df['country'] = df['country'].apply(lambda x:re.sub('[^\x00-\x7f]', '', x))

In [35]:
df

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,processed_text,country
0,1340539111971516416,Rachel Roh,La Crescenta-Montrose CA,"Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False,same folks said daikon paste could treat a cytokine storm,United States
1,1338158543359250433,Albert Fong,San Francisco CA,"Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,Twitter Web App,1,1,False,while the world has been on the wrong side of history this year hopefully the biggest vaccination effort weve ev,United States
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False,russian vaccine is created to last years,Your Bed
3,1337855739918835717,Charles Adler,Vancouver BC - Canada,"Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",NaN,Twitter Web App,446,2129,False,facts are immutable senator even when youre not ethically sturdy enough to acknowledge them you were born i,Canada
4,1337854064604966912,Citizen News Channel,nan,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False,explain to me again why we need a vaccine,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228202,1460170772299665408,VaxBLR,Bengaluru India,"Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 09:00:15,45+ #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:30PM\nFREE Slots 0\nPAID Slots \n#COVISHIELD - Dose1:… https://t.co/F0WeHxW7oe,"['URBAN', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,0,False,availability for at pm free slots paid slots dose,India
228203,1460163268262051841,VaxBLR,Bengaluru India,"Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:26,"18-44 #BBMP #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFREE Slots \n#COVISHIELD - Dose1:1676, Dose2:… https://t.co/3k8SHHcVVd","['BBMP', 'Bengaluru', 'CovidVaccine', 'COVISHIELD']",VaxBlr,0,1,False,availability for at pm free slots dose dose,India
228204,1460163224221851655,VaxBLR,Bengaluru India,"Hourly updates on FREE and PAID 18+ and 45+ vaccine slot availability across #Bengaluru BBMP,URBAN & RURAL",2021-06-21 08:44:34,31,0,0,False,2021-11-15 08:30:15,18-44 #URBAN #Bengaluru #CovidVaccine Availability for 15/11 at 02:00PM\nFR

In [41]:

loc_count = dict(df['country'].value_counts(dropna=True))
print('val', dict(df['country'].value_counts(dropna=True)))
for key, val in loc_count.items():
    print(f'{key}:{val}')


val {'nan': 66911, 'India': 53937, 'United States': 22292, 'Canada': 8510, 'United Kingdom': 5726, 'China': 2593, 'Pakistan': 2150, 'Sri Lanka': 1951, 'Philippines': 1585, '': 1560, 'Australia': 1477, 'Malaysia': 1200, 'Russia': 1029, 'Germany': 1015, 'London': 983, 'Ireland': 928, 'United Arab Emirates': 920, 'France': 870, 'South Africa': 845, ' ': 801, 'Thailand': 720, 'Bangladesh': 575, 'Kenya': 570, 'World': 549, 'Japan': 510, 'Estados Unidos': 508, 'Singapore': 496, 'Belgium': 476, 'Hong Kong': 463, 'Turkey': 461, 'Italy': 438, 'Qatar': 390, '  ': 358, 'New York': 266, 'Switzerland': 260, 'US of Florida': 243, 'Los Angeles': 238, 'Argentina': 236, 'World Wide Web': 215, 'Indonesia': 210, 'US of Ohio': 195, 'San Jose': 195, 'Nigeria': 192, 'Brazil': 176, 'StocksLand ': 174, 'Korea': 168, 'New York City': 168, 'US of Colorado': 167, 'Lebanon': 164, 'Denmark': 160, '   ': 159, 'Europe': 158, 'Southern Cal': 156, 'Trincomalee Sri Lanka': 154, 'T -': 152, 'Petaling Jaya': 149, 'Everyw

In [38]:
df.sort_values(by='date', ascending=True)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,processed_text,country
44,1337727767551553536,Daily News Egypt,Egypt,Egypt's Only Daily Independent Newspaper in English,2009-04-26 07:56:24,278080,116,765,True,2020-12-12 11:55:28,#FDA authorizes #PfizerBioNTech #coronavirus vaccine for emergency use\nhttps://t.co/VZ6bDPbbPG,"['FDA', 'PfizerBioNTech', 'coronavirus']",Twitter Web App,1,1,False,authorizes vaccine for emergency use,Egypt
43,1337728701820268544,City A.M.,London England,"London's business newspaper - News, Opinion, and Analysis. For all distribution queries, see @CityAMDist.",2009-06-09 13:53:06,66224,603,771,True,2020-12-12 11:59:10,Trump announces #vaccine rollout 'in less than 24 hours'\n\nThe first Americans will be vaccinated against… https://t.co/6379SZjNQi,['vaccine'],Twitter Web App,0,2,False,trump announces rollout in less than hours the first americans will be vaccinated against,United Kingdom
42,1337732077278662657,mystic,nan,I believe in the goodness of humanity.,2020-05-20 10:57:00,14,162,1341,False,2020-12-12 12:12:35,#CovidVaccine #Pfizer-BioNTech COVID-19 #PfizerBioNTech #COVID19 #CapeTown https://t.co/uCpwv0S4nl,"['CovidVaccine', 'Pfizer', 'PfizerBioNTech', 'COVID19', 'CapeTown']",Twitter for iPhone,0,1,False,biontech covid,nan
41,1337732996380696578,KHuntson,USA,"RN, health educator, writer, researcher, wife, mom, truth-seeker. Covid-19 is a crime against humanity. There is no such thing as a risk-free world.",2014-01-31 01:25:24,2035,1672,6734,False,2020-12-12 12:16:14,FACT SHEET FOR RECIPIENTS AND CAREGIVERS\n\n#PfizerBioNTech \n#CovidVaccine \n\nhttps://t.co/V0mn08VTwj,"['PfizerBioNTech', 'CovidVaccine']",Twitter Web App,0,0,False,fact sheet for recipients and caregivers,United States
40,1337733048914358272,"Party politics isn't working for anyone, anywhere",Wales United Kingdom,"No lie to another is respectful; no lie to yourself is healthy. ""Official lies"" should mean ""Official jail""",2016-08-03 20:11:00,761,504,33675,False,2020-12-12 12:16:27,@mirekR9 @sputnikvaccine @AstraZeneca it's the only one which doesn't have any side effects \n\nthe #PfizerBioNTech a… https://t.co/IWELAATAmd,['PfizerBioNTech'],Twitter Web App,0,0,False,its the only one which doesnt have any side effects the a,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225292,1463249661683290118,TexasNative,nan,NaN,2021-05-20 19:37:03,0,7,179,False,2021-11-23 20:54:39,@snehamordani @BharatBiotech Eagerly anticipating! #COVAXIN,['COVAXIN'],Twitter Web App,0,0,False,eagerly anticipating,nan
223548,1463249967452246025,UNSCARED,In The Matrix,"Quos Deus vult perdere, prius dementat.\n\nNo weapons, no matter how powerful, can help\nthe West until it overcomes its loss of willpower.\n#pureblood",2021-01-28 01:21:32,210,16,4107,False,2021-11-23 20:55:52,Σｔｅｒｎｉｔｙ ｔｏ Ｆｌａｔｔｅｎ ｔｈｅ Ｃｕｒｖｅ ... \n\n😂😂😂 funny but true!\n\n#Pfizer\n#Moderna\n#johnsonandjohnson\n#AstraZeneca… https://t.co/Btg9b4icjR,"['Pfizer', 'Moderna', 'johnsonandjohnson', 'AstraZeneca']",Twitter Web App,0,0,False,σｔｅｒｎｉｔｙ ｔｏ ｆｌａｔｔｅｎ ｔｈｅ ｃｕｒｖｅ funny but true,In The Matrix
225291,1463250299406077956,"Dr. A. K. Singh; MD, DM (Endo) 🇮🇳",Kolkata India,"Endocrinologist, GD Hosp. & Diab Institute, Kolkata (India). \nAd. Editor- Williams Textbook of Endocrinology 14th Ed'2020. \nAsso. Editor- IJDDC, Word J Diabetes",2018-12-13 11:38:45,4643,194,5166,False,2021-11-23 20:57:11,A case control study from India found vaccine effectiveness of 50% with 2 doses of #Covaxin for delta strain! Just… https://t.co/Yb9ysbqBe9,['Covaxin'],Twitter for iPhone,0,1,False,a case control study from india found vaccine effectiveness of with doses of for delta strain just,India
223547,1463250477928394755,globalEDGE,Michigan State University MI,"Your collection of resources in international business, global trade, stats, cross cultural advice 